## How to build artificial neural networks with Keras and TensorFlow
Dataset is obtained from [Kaggle](https://www.kaggle.com/datasets/teejmahal20/airline-passenger-satisfaction?select=train.csv).

Click the image below to read the post online.

<a target="_blank" href="https://www.machinelearningnuggets.com/how-to-build-artificial-neural-networks-with-keras-and-tensorflow
"><img src="https://digitalpress.fra1.cdn.digitaloceanspaces.com/mhujhsj/2022/07/logo.png" alt="Open in ML Nuggets"></a>

# TensorFlow basics

## Tensors

TensorFlow operates on multidimensional arrays or tensors represented as tf.Tensor objects.

In [ ]:
import tensorflow as tf
import numpy as np

x = tf.constant([[7., 8., 9.],
                 [10., 11., 12.]])

print(x)
print(x.shape)
print(x.dtype)

In [ ]:
x[0]

In [ ]:
x[1:4]

In [ ]:
x**2

In [ ]:
x @ tf.transpose(x)

In [ ]:
tf.concat([x, x, x], axis=0)

In [ ]:
np.array(x)

In [ ]:
x.numpy()

In [ ]:
x.shape.as_list()

In [ ]:
tf.nn.softmax(x, axis=-1)

In [ ]:
tf.reduce_sum(x)

In [ ]:
if tf.config.list_physical_devices('GPU'):
  print("TensorFlow **IS** using the GPU")
else:
  print("TensorFlow **IS NOT** using the GPU")

In [ ]:
# There can be an arbitrary number of
# axes (sometimes called "dimensions")
rank_3_tensor = tf.constant([
  [[0, 1, 2, 3, 4],
   [5, 6, 7, 8, 9]],
  [[10, 11, 12, 13, 14],
   [15, 16, 17, 18, 19]],
  [[20, 21, 22, 23, 24],
   [25, 26, 27, 28, 29]],])

print(rank_3_tensor)

In [ ]:
# If you have three string tensors of different lengths, this is OK.
tensor_of_strings = tf.constant(["Gray wolf",
                                 "Quick brown fox",
                                 "Lazy dog"])
# Note that the shape is (3,). The string length is not included.
print(tensor_of_strings)

In [ ]:
ragged_list = [
    [0, 1, 2, 3],
    [4, 5],
    [6, 7, 8],
    [9]]

In [ ]:
try:
  tensor = tf.constant(ragged_list)
except Exception as e:
  print(f"{type(e).__name__}: {e}")

In [ ]:
ragged_tensor = tf.ragged.constant(ragged_list)
print(ragged_tensor)

In [ ]:
print(ragged_tensor.shape)

In [ ]:
# Sparse tensors store values by index in a memory-efficient manner
sparse_tensor = tf.sparse.SparseTensor(indices=[[0, 0], [1, 2]],
                                       values=[1, 2],
                                       dense_shape=[3, 4])
print(sparse_tensor, "\n")

# You can convert sparse tensors to dense
print(tf.sparse.to_dense(sparse_tensor))

## Variables

In [ ]:

# Uncomment to see where your variables get placed (see below)
tf.debugging.set_log_device_placement(True)

In [ ]:
my_tensor = tf.constant([[8.0, 8.0], [6.0, 5.0]])
my_variable = tf.Variable(my_tensor)

# Variables can be all kinds of types, just like tensors
bool_variable = tf.Variable([True, False, False, True])
complex_variable = tf.Variable([8 + 4j, 8 + 1j])

In [ ]:
my_variable

In [ ]:
print("Shape: ", my_variable.shape)
print("DType: ", my_variable.dtype)
print("As NumPy: ", my_variable.numpy())

In [ ]:
print("A variable:", my_variable)
print("\nViewed as a tensor:", tf.convert_to_tensor(my_variable))
print("\nIndex of highest value:", tf.math.argmin(my_variable))

# This creates a new tensor; it does not reshape the variable.
print("\nCopying and reshaping: ", tf.reshape(my_variable, [1,4]))

In [ ]:
a = tf.Variable([2.0, 3.0])
# This will keep the same dtype, float32
a.assign([1, 2]) 
# Not allowed as it resizes the variable: 
try:
  a.assign([1.0, 2.0, 3.0])
except Exception as e:
  print(f"{type(e).__name__}: {e}")

In [ ]:
a = tf.Variable([2.0, 3.0])
# Create b based on the value of a
b = tf.Variable(a)
a.assign([5, 6])

# a and b are different
print(a.numpy())
print(b.numpy())

# There are other versions of assign
print(a.assign_add([2,3]).numpy())  # [7. 9.]
print(a.assign_sub([7,9]).numpy())  # [0. 0.]

In [ ]:
# Create a and b; they will have the same name but will be backed by
# different tensors.
a = tf.Variable(my_tensor, name="ml nuggets")
# A new variable with the same name, but different value
# Note that the scalar add is broadcast
b = tf.Variable(my_tensor + 1, name="ml nuggets")

# These are elementwise-unequal, despite having the same name
print(a == b)

In [ ]:
step_counter = tf.Variable(1, trainable=False)

In [ ]:
with tf.device('CPU:0'):

  # Create some tensors
  a = tf.Variable([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
  b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
  c = tf.matmul(a, b)

print(c)

In [ ]:
with tf.device('CPU:0'):
  a = tf.Variable([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
  b = tf.Variable([[1.0, 2.0, 3.0]])

with tf.device('GPU:0'):
  # Element-wise multiply
  k = a * b

print(k)

## Automatic differentiation

In [ ]:
x = tf.Variable(47.0)

with tf.GradientTape() as tape:
  y = x**2

In [ ]:
# dy = 2x * dx
dy_dx = tape.gradient(y, x)
dy_dx.numpy()

In [ ]:
w = tf.Variable(tf.random.normal((3, 2)), name='w')
b = tf.Variable(tf.zeros(2, dtype=tf.float32), name='b')
x = [[1., 2., 3.]]

with tf.GradientTape(persistent=True) as tape:
  y = x @ w + b
  loss = tf.reduce_mean(y**2)

In [ ]:
[dl_dw, dl_db] = tape.gradient(loss, [w, b])

In [ ]:
print(w.shape)
print(dl_dw.shape)

In [ ]:
my_vars = {
    'w': w,
    'b': b
}

grad = tape.gradient(loss, my_vars)
grad['b']

In [ ]:
layer = tf.keras.layers.Dense(2, activation='relu')
x = tf.constant([[1., 2., 3.]])

with tf.GradientTape() as tape:
  # Forward pass
  y = layer(x)
  loss = tf.reduce_mean(y**2)

# Calculate gradients with respect to every trainable variable
grad = tape.gradient(loss, layer.trainable_variables)

In [ ]:
for var, g in zip(layer.trainable_variables, grad):
  print(f'{var.name}, shape: {g.shape}')

## Graphs & functions

In [ ]:
import timeit
from datetime import datetime

In [ ]:
# Define a Python function.
def a_regular_function(x, y, b):
  x = tf.matmul(x, y)
  x = x + b
  return x

# `a_function_that_uses_a_graph` is a TensorFlow `Function`.
a_function_that_uses_a_graph = tf.function(a_regular_function)

# Make some tensors.
x1 = tf.constant([[1.0, 2.0]])
y1 = tf.constant([[2.0], [3.0]])
b1 = tf.constant(4.0)

orig_value = a_regular_function(x1, y1, b1).numpy()
# Call a `Function` like a Python function.
tf_function_value = a_function_that_uses_a_graph(x1, y1, b1).numpy()
assert(orig_value == tf_function_value)

In [ ]:
def inner_function(x, y, b):
  x = tf.matmul(x, y)
  x = x + b
  return x

# Use the decorator to make `outer_function` a `Function`.
@tf.function
def outer_function(x):
  y = tf.constant([[2.0], [3.0]])
  b = tf.constant(4.0)

  return inner_function(x, y, b)

# Note that the callable will create a graph that
# includes `inner_function` as well as `outer_function`.
outer_function(tf.constant([[1.0, 2.0]])).numpy()

In [ ]:
def simple_relu(x):
  if tf.greater(x, 0):
    return x
  else:
    return 0

# `tf_simple_relu` is a TensorFlow `Function` that wraps `simple_relu`.
tf_simple_relu = tf.function(simple_relu)

print("First branch, with graph:", tf_simple_relu(tf.constant(1)).numpy())
print("Second branch, with graph:", tf_simple_relu(tf.constant(-1)).numpy())

In [ ]:
# This is the graph-generating output of AutoGraph.
print(tf.autograph.to_code(simple_relu))

In [ ]:
# This is the graph itself.
print(tf_simple_relu.get_concrete_function(tf.constant(1)).graph.as_graph_def())

## Modules, layers, and models

In [ ]:

from datetime import datetime

%load_ext tensorboard

In [ ]:
class SimpleModule(tf.Module):
  def __init__(self, name=None):
    super().__init__(name=name)
    self.a_variable = tf.Variable(5.0, name="train_me")
    self.non_trainable_variable = tf.Variable(5.0, trainable=False, name="do_not_train_me")
  def __call__(self, x):
    return self.a_variable * x + self.non_trainable_variable

simple_module = SimpleModule(name="simple")

simple_module(tf.constant(5.0))

In [ ]:
# All trainable variables
print("trainable variables:", simple_module.trainable_variables)
# Every variable
print("all variables:", simple_module.variables)

In [ ]:
class Dense(tf.Module):
  def __init__(self, in_features, out_features, name=None):
    super().__init__(name=name)
    self.w = tf.Variable(
      tf.random.normal([in_features, out_features]), name='w')
    self.b = tf.Variable(tf.zeros([out_features]), name='b')
  def __call__(self, x):
    y = tf.matmul(x, self.w) + self.b
    return tf.nn.relu(y)

In [ ]:
class SequentialModule(tf.Module):
  def __init__(self, name=None):
    super().__init__(name=name)

    self.dense_1 = Dense(in_features=3, out_features=3)
    self.dense_2 = Dense(in_features=3, out_features=2)

  def __call__(self, x):
    x = self.dense_1(x)
    return self.dense_2(x)

# You have made a model!
my_model = SequentialModule(name="the_model")

# Call it, with random results
print("Model results:", my_model(tf.constant([[2.0, 2.0, 2.0]])))

In [ ]:
class MySequentialModule(tf.Module):
  def __init__(self, name=None):
    super().__init__(name=name)

    self.dense_1 = Dense(in_features=3, out_features=3)
    self.dense_2 = Dense(in_features=3, out_features=2)

  @tf.function
  def __call__(self, x):
    x = self.dense_1(x)
    return self.dense_2(x)

# You have made a model with a graph!
my_model = MySequentialModule(name="the_model")

In [ ]:
# Set up logging.
stamp = datetime.now().strftime("%Y%m%d-%H%M%S")
logdir = "logs/func/%s" % stamp
writer = tf.summary.create_file_writer(logdir)

# Create a new model to get a fresh trace
# Otherwise the summary will not see the graph.
new_model = MySequentialModule()

# Bracket the function call with
# tf.summary.trace_on() and tf.summary.trace_export().
tf.summary.trace_on(graph=True)
tf.profiler.experimental.start(logdir)
# Call only one tf.function when tracing.
z = print(new_model(tf.constant([[2.0, 2.0, 2.0]])))
with writer.as_default():
  tf.summary.trace_export(
      name="my_func_trace",
      step=0,
      profiler_outdir=logdir)

In [ ]:
%tensorboard --logdir logs/func

## Training loops

In [ ]:
import matplotlib.pyplot as plt

colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

In [ ]:
# The actual line
TRUE_W = 3.0
TRUE_B = 2.0

NUM_EXAMPLES = 201

# A vector of random x values
x = tf.linspace(-2,2, NUM_EXAMPLES)
x = tf.cast(x, tf.float32)

def f(x):
  return x * TRUE_W + TRUE_B

# Generate some noise
noise = tf.random.normal(shape=[NUM_EXAMPLES])

# Calculate y
y = f(x) + noise

In [ ]:
# Plot all the data
plt.plot(x, y, '.')
plt.show()

In [ ]:
class MyModel(tf.Module):
  def __init__(self, **kwargs):
    super().__init__(**kwargs)
    # Initialize the weights to `5.0` and the bias to `0.0`
    # In practice, these should be randomly initialized
    self.w = tf.Variable(5.0)
    self.b = tf.Variable(0.0)

  def __call__(self, x):
    return self.w * x + self.b

model = MyModel()

# List the variables tf.modules's built-in variable aggregation.
print("Variables:", model.variables)

# Verify the model works
assert model(3.0).numpy() == 15.0

In [ ]:
# This computes a single loss value for an entire batch
def loss(target_y, predicted_y):
  return tf.reduce_mean(tf.square(target_y - predicted_y))

In [ ]:
plt.plot(x, y, '.', label="Data")
plt.plot(x, f(x), label="Ground truth")
plt.plot(x, model(x), label="Predictions")
plt.legend()
plt.show()

print("Current loss: %1.6f" % loss(y, model(x)).numpy())

In [ ]:
# Given a callable model, inputs, outputs, and a learning rate...
def train(model, x, y, learning_rate):

  with tf.GradientTape() as t:
    # Trainable variables are automatically tracked by GradientTape
    current_loss = loss(y, model(x))

  # Use GradientTape to calculate the gradients with respect to W and b
  dw, db = t.gradient(current_loss, [model.w, model.b])

  # Subtract the gradient scaled by the learning rate
  model.w.assign_sub(learning_rate * dw)
  model.b.assign_sub(learning_rate * db)

In [ ]:
model = MyModel()

# Collect the history of W-values and b-values to plot later
weights = []
biases = []
epochs = range(10)

# Define a training loop
def report(model, loss):
  return f"W = {model.w.numpy():1.2f}, b = {model.b.numpy():1.2f}, loss={loss:2.5f}"


def training_loop(model, x, y):

  for epoch in epochs:
    # Update the model with the single giant batch
    train(model, x, y, learning_rate=0.1)

    # Track this before I update
    weights.append(model.w.numpy())
    biases.append(model.b.numpy())
    current_loss = loss(y, model(x))

    print(f"Epoch {epoch:2d}:")
    print("    ", report(model, current_loss))

In [ ]:
current_loss = loss(y, model(x))

print(f"Starting:")
print("    ", report(model, current_loss))

training_loop(model, x, y)

In [ ]:
plt.plot(epochs, weights, label='Weights', color=colors[0])
plt.plot(epochs, [TRUE_W] * len(epochs), '--',
         label = "True weight", color=colors[0])

plt.plot(epochs, biases, label='bias', color=colors[1])
plt.plot(epochs, [TRUE_B] * len(epochs), "--",
         label="True bias", color=colors[1])

plt.legend()
plt.show()

In [ ]:
plt.plot(x, y, '.', label="Data")
plt.plot(x, f(x), label="Ground truth")
plt.plot(x, model(x), label="Predictions")
plt.legend()
plt.show()

print("Current loss: %1.6f" % loss(model(x), y).numpy())

In [ ]:
class MyModelKeras(tf.keras.Model):
  def __init__(self, **kwargs):
    super().__init__(**kwargs)
    # Initialize the weights to `5.0` and the bias to `0.0`
    # In practice, these should be randomly initialized
    self.w = tf.Variable(5.0)
    self.b = tf.Variable(0.0)

  def call(self, x):
    return self.w * x + self.b

keras_model = MyModelKeras()

# Reuse the training loop with a Keras model
training_loop(keras_model, x, y)

# You can also save a checkpoint using Keras's built-in support
keras_model.save_weights("my_checkpoint")

In [ ]:
keras_model = MyModelKeras()

# compile sets the training parameters
keras_model.compile(
    # By default, fit() uses tf.function().  You can
    # turn that off for debugging, but it is on now.
    run_eagerly=False,

    # Using a built-in optimizer, configuring as an object
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.1),

    # Keras comes with built-in MSE error
    # However, you could use the loss function
    # defined above
    loss=tf.keras.losses.mean_squared_error,
)

In [ ]:
print(x.shape[0])
keras_model.fit(x, y, epochs=10, batch_size=1000)

# How to train an artificial neural network with TensorFlow

## Data pre-processing

In [ ]:
!git clone https://github.com/mlnuggets/tensorflow.git
!mv tensorflow/train.csv train.csv 

In [ ]:
import pandas as pd
df = pd.read_csv("train.csv")

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df['Arrival Delay in Minutes'] = df['Arrival Delay in Minutes'].mean()

In [ ]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
df = df.assign(satisfaction = labelencoder.fit_transform(df["satisfaction"]))

In [ ]:
categories = df.select_dtypes(include=['object']).columns.tolist()

In [ ]:
categories

## Data transformation

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
X = df.drop(["Unnamed: 0", "id","satisfaction"], axis=1)
y = df["satisfaction"]
random_state = 13
test_size = 0.3
transformer = ColumnTransformer(transformers=[('cat', OneHotEncoder(handle_unknown='ignore', drop="first"), categories)],remainder=MinMaxScaler())
X = transformer.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size,random_state=random_state)

In [ ]:
X_train

In [ ]:
X_train.shape[1]

## Build the artificial neural network

In [ ]:
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import Input
import tensorflow as tf

In [ ]:
model = Sequential(
    [
        Input(shape=(X_train.shape[1],)),
        Dense(64, activation="relu",  kernel_initializer="glorot_uniform",name="layer1"),
        Dense(32, activation="relu", kernel_initializer="glorot_uniform", name="layer2"),
        Dense(1, activation="sigmoid", name="layer3"),
    ]
)

In [ ]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
history = model.fit(X_train, y_train, validation_data =(X_test, y_test), batch_size = 32, epochs = 10)

In [ ]:
history

## Visualize model performance

In [ ]:
metrics_df = pd.DataFrame(history.history)
metrics_df[["loss","val_loss"]].plot();
metrics_df[["accuracy","val_accuracy"]].plot();

## Add Dropout Regularization to fight over-fitting

In [ ]:
from tensorflow.keras.layers import BatchNormalization

from tensorflow.keras.layers import Dropout
model = Sequential(
    [
        Input(shape=(X_train.shape[1],)),
        Dense(64, activation="relu",  kernel_initializer="glorot_uniform",name="layer1"),
        BatchNormalization(),
        Dropout(rate=0.1),
        Dense(32, activation="relu", kernel_initializer="glorot_uniform", name="layer2"),
        Dense(1, activation="sigmoid", name="layer3"),
    ])
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
history = model.fit(X_train, y_train, validation_data =(X_test, y_test), batch_size = 32, epochs = 10)

# How to accelerate training with Batch normalization

In [ ]:
model = Sequential(
    [
        Input(shape=(X_train.shape[1],)),
        Dense(64, activation="relu",  kernel_initializer="glorot_uniform",name="layer1"),
        BatchNormalization(),
        Dense(32, activation="relu", kernel_initializer="glorot_uniform", name="layer2"),
        Dense(1, activation="sigmoid", name="layer3"),
    ]
)
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
history = model.fit(X_train, y_train, validation_data =(X_test, y_test), batch_size = 32, epochs = 10)

## How to stop model training at the right time with Early Stopping

In [ ]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
callbacks = [tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)]
history = model.fit(X_train, y_train, validation_data =(X_test, y_test), batch_size = 32, epochs = 10,callbacks=callbacks)

## How to save the best model with checkpoints

In [ ]:
checkpoint_filepath = "model_checkpoint"

In [ ]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
callbacks = [
             
             tf.keras.callbacks.EarlyStopping(monitor="loss", patience=3),
             
             tf.keras.callbacks.ModelCheckpoint(
             filepath=checkpoint_filepath,
             save_weights_only=True,
             monitor="val_accuracy",
             mode="max",
             save_best_only=True)]

history = model.fit(X_train, y_train, validation_data =(X_test, y_test), batch_size = 32, epochs = 10,callbacks=callbacks)

In [ ]:
# The model weights (that are considered the best) are loaded into the model.
model.load_weights(checkpoint_filepath)

## Make predictions on the test set

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
y_pred = (y_pred > 0.5)

## Plot the confusion matrix

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

## Make a single prediction

In [ ]:
import numpy as np
test_data = np.expand_dims(X_test[0], axis=0)

In [ ]:
model.predict(test_data) > 0.5

## How to save and load Keras models

In [ ]:
# Save the weights
model.save_weights('./checkpoints/my_checkpoint')

In [ ]:
model.save("saved_model")

In [ ]:
new_model = tf.keras.models.load_model('saved_model')
new_model.summary()

## How to perform K-fold cross-validation on Keras models 

In [ ]:
pip install scikeras[tensorflow]

In [ ]:
# https://www.adriangb.com/scikeras/stable/
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import cross_val_score

In [ ]:
def make_model():
  model = Sequential([
        Input(shape=(X_train.shape[1],)),
        Dense(64, activation="relu",  kernel_initializer="glorot_uniform",name="layer1"),
        Dense(32, activation="relu", kernel_initializer="glorot_uniform", name="layer2"),
        Dense(1, activation="sigmoid", name="layer3"), ])
  return model

In [ ]:
model = KerasClassifier(model=make_model, batch_size=32, optimizer="adam", metrics=["accuracy"],loss="binary_crossentropy",validation_split=0.2, epochs=1)

In [ ]:
accuracies = cross_val_score(estimator=model, X=X_train, y=y_train, cv = 10, n_jobs = -1)

In [ ]:
mean = accuracies.mean()
mean

In [ ]:
variance = accuracies.var()
variance

## How to tune model hyperparameters in Keras

In [ ]:
from sklearn.model_selection import GridSearchCV
def make_model():
  model = Sequential([
        Input(shape=(X_train.shape[1],)),
        Dense(64, activation="relu",  kernel_initializer="glorot_uniform",name="layer1"),
        Dropout(rate=0.1),
        Dense(32, activation="relu", kernel_initializer="glorot_uniform", name="layer2"),
        Dense(1, activation="sigmoid", name="layer3"),])
  return model

In [ ]:
model = KerasClassifier(model=make_model, metrics=["accuracy"],loss="binary_crossentropy",validation_split=0.2, epochs=1)

In [ ]:
params = {
    "batch_size":[10,20,32,64],
    "epochs":[2,3,4],
    "optimizer":["adam","rmsprop"]
}

In [ ]:
grid_search = GridSearchCV(estimator=model,
                           param_grid=params,
                           scoring="accuracy",
                           cv=2)

In [ ]:
grid_search = grid_search.fit(X_train,y_train)

In [ ]:
best_param = grid_search.best_params_
best_accuracy = grid_search.best_score_

In [ ]:
best_param

In [ ]:
best_accuracy

## How to tune the network parameters

In [ ]:
def make_clf(hidden_layer_sizes, dropout):
    model = Sequential()
    model.add(Input(shape=(X_train.shape[1],)))
    for hidden_layer_size in hidden_layer_sizes:
        model.add(Dense(hidden_layer_size, activation="relu"))
        model.add(Dropout(dropout))
    model.add(Dense(1, activation="sigmoid"))
    return model

In [ ]:
my_model = KerasClassifier(
    model=make_clf,
    loss="binary_crossentropy",
    optimizer="adam",
    optimizer__learning_rate=0.1,
    model__hidden_layer_sizes=(100,),
    model__dropout=0.5,
    verbose=False,
)

In [ ]:
params = {
    'optimizer__learning_rate': [0.05, 0.1],
    'model__hidden_layer_sizes': [(100, ), (50, 50, )],
    'model__dropout': [0, 0.5],
}

gs = GridSearchCV(my_model, params, scoring='accuracy', n_jobs=-1, verbose=True)

gs.fit(X_train, y_train)

print(gs.best_score_, gs.best_params_)

## Where to go from here
Follow us on [LinkedIn](https://www.linkedin.com/company/mlnuggets), [Twitter](https://twitter.com/ml_nuggets), [GitHub](https://github.com/mlnuggets) and subscribe to our [blog](https://www.machinelearningnuggets.com/#/portal) so that you don't miss a new issue.